<a href="https://colab.research.google.com/github/A00827038/ModelacionAgentes/blob/main/M1_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad 1

En esta actividad modelaremos el comportamiento de agentes reactivos simples–robots de limpieza y piso.


## Reglas

Dado
1. Habitación de MxN espacios.
2. Número de agentes.
3. Porcentaje de celdas inicialmente sucias.
4. Tiempo máximo de ejecución.

Realizar lo Siguiente
1. Inicializar las celdas sucias (ubicaciones aleatorias).
2. Todos los agentes empiezan en la celda [1,1].
3. Los agentes recorreran el grid y limpiar las celdas sucias. 

*En cada paso de tiempo:
1. Si la celda está sucia, entonces aspira.
2. Si la celda está limpia, el agente elije una dirección aleatoria para moverse (unas de las 8 celdas vecinas) 

## Imports

Antes de empezar a crear el modelo del juego de la vida con multiagentes es necesario tener instalado los siguientes paquetes:
- `python`: asegúrense de usar la versión 3+.
- `mesa`: el framework de Python para el modelado de agentes.
- `numpy`: es una biblioteca de Python para el manejo de matrices, arreglos, manipulación matemática, lógica y mucho más.
- `matplotlib`: es una biblioteca para crear visualizaciones estáticas, animadas e interactivas en Python.

Para poder modelar el juego de la vida usando el framework de `mesa` es necesario importar dos clases: una para el modelo general, y otro para los agentes. 

In [6]:
!pip3 install mesa

     |████████████████████████████████| 668 kB 15.0 MB/s 
     |████████████████████████████████| 60 kB 5.7 MB/s 


In [49]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `SingleGrid` que fuerza un solo objeto por celda.
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

## Crear el modelo

Antes que nada el presente modelo se encuentra basado en el [tutorial introductorio](https://mesa.readthedocs.io/en/master/tutorials/intro_tutorial.html). Lo modifiqué un poco para que funcionara para el presente problema pero en esencia es lo mismo.

In [81]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid. Si hay un agente en la celda, entonces se asigna un valor
    de 2. Si el piso está sucio, entonces se le asigna un valor de 1. Si el piso está limpio, entonces el valor es 0. 
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
          if isinstance(content, robotAgent):
            grid[x][y] = 2
          elif isinstance(content,floorAgent):
              grid[x][y] = 1
          else:
            grid[x][y] = 0

    
    return grid

class robotAgent(Agent):
    '''
    Un robot que se mueve y limpia el piso.
    '''
    def __init__(self, unique_id, pos, model):
        super().__init__(unique_id, pos, model)
        self.next_state = None
    
    def step(self):
        '''
        Método que limpia, y mueve al robot a otra de sus 8 celdas vecinas.
        '''

        #Possibles lugares a moverse
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        
        #Lugar a donde se va a mover
        self.next_state = self.random.choice(possible_steps)

        #Si hay suciedad, entonces la elimina.
        x, y = self.pos
        this_cell = self.model.grid.get_cell_list_contents([self.pos])
        floor = [obj for obj in this_cell if isinstance(obj, floorAgent)][0]
        if floor:
            # Elimina la suciedad
            self.model.grid._remove_agent(self.pos, floor)
            self.model.schedule.remove(floor)
       
    def advance(self):
      self.model.grid.move_agent(self, self.next_state)
        

class floorAgent(Agent):
    '''
    Representa a una celda con estado sucio.
    '''
    def __init__(self, unique_id, pos, model):
        '''
        Crea un agente de piso sucio, también se le asigna un identificador 
        formado por una tupla (x,y). También se define un nuevo estado cuyo valor será definido por las 
        reglas mencionadas arriba.
        '''
        super().__init__(unique_id, pos, model)
        self.sucio = 1;
        
    # def step(self):
    #     '''
    #     Este método es el que calcula si la celda se limpiará dependiendo si un agente robot se encuentra ahí.
    #     El estado sucio no se cambia aquí, se almacena en self.next_state. La idea 
    #     es esperar a que todos los agentes calculen su estado y una vez hecho eso, ya hacer el cambio.
    #     '''  
        
    #     #Obtiene los agentes dentro de la celda
    #     x, y = self.pos
    #     this_cell = self.model.grid.get_cell_list_contents([self.pos])
    #     robots = [obj for obj in this_cell if isinstance(obj, robotAgent)]
  
    #     #Checa si la celda está sucia
    #     #Si hay robots, entonces la celda se limpia
    #     self.next_state = self.sucio
    #     if self.next_state == 0:
    #         if robots:
    #             self.next_state = 1
            
    # def advance(self):
    #     '''
    #     Define el nuevo estado calculado del método step.
    #     '''
    #     self.sucio = self.next_state
            
class CleanFloorModel(Model):
    '''
    Define el modelo de limpia piso con robots.
    '''
    def __init__(self, width, height, N, CS):
        self.num_robots = N
        self.num_CS = CS
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        
        #Creamos los robots
        for i in range (self.num_robots):
            robot = robotAgent(i,(1,1),self)
            self.grid.place_agent(robot, (1, 1))
            self.schedule.add(robot)

        #Creamos el piso sucio
        for n in (self.num_CS):
            # Add the agent to a random grid cell
            x = self.random.randrange(width)
            y = self.random.randrange(height)
            floor = floorAgent(n+N,(x,y),self)
            self.grid.place_agent(floor, (x, y))
            self.schedule.add(floor)
        
        # Aquí definimos con colector para obtener el grid completo. Aquí recompilamos la información.
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
    
    def step(self):
        '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
        '''
        self.datacollector.collect(self)
        self.schedule.step()

A continuación corremos el modelo

In [79]:
# Definimos el tamaño del Grid
width = 10
height = 10

# Definimos el número de robots
N = 5

# Definimos el porcentaje de celdas sucias
PORCENTAJE = 30

# Definimos las celdas sucias
CS = width*height*PORCENTAJE*0.01
print(CS)

# Definimos el tiempo máximo
TIEMPO_MAX = 50

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
model = CleanFloorModel(width, height, N, CS)
for i in range(TIEMPO_MAX):
    model.step()

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

30.0


TypeError: ignored

Obtenemos la información que almacenó el colector, este nos entregará un DataFrame de pandas que contiene toda la información.

In [65]:
all_grid = model.datacollector.get_model_vars_dataframe()

NameError: ignored

Graficamos la información usando `matplotlib`

In [129]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.binary)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=TIEMPO_MAX)

In [130]:
anim

## CUDA

El paso siguiente será modificar el modelo anterior para que funcione con CUDA

#Conclusiones